### 🚀 For an interactive experience, head over to our [demo platform](https://var.vision/demo) and dive right in! 🌟

In [1]:
################## 1. Download checkpoints and build models
import os
import os.path as osp
import torch, torchvision
import random
import numpy as np
import PIL.Image as PImage, PIL.ImageDraw as PImageDraw
setattr(torch.nn.Linear, 'reset_parameters', lambda self: None)     # disable default parameter init for faster speed
setattr(torch.nn.LayerNorm, 'reset_parameters', lambda self: None)  # disable default parameter init for faster speed
from models import VQVAE, build_vae_var

MODEL_DEPTH = 16    # TODO: =====> please specify MODEL_DEPTH <=====
assert MODEL_DEPTH in {16, 20, 24, 30}


# download checkpoint
hf_home = 'https://huggingface.co/FoundationVision/var/resolve/main'
vae_ckpt, var_ckpt = 'vae_ch160v4096z32.pth', f'var_d{MODEL_DEPTH}.pth'
if not osp.exists(vae_ckpt): os.system(f'wget {hf_home}/{vae_ckpt}')
if not osp.exists(var_ckpt): os.system(f'wget {hf_home}/{var_ckpt}')

# build vae, var
patch_nums = (1, 2, 3, 4, 5, 6, 8, 10, 13, 16)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if 'vae' not in globals() or 'var' not in globals():
    vae, var = build_vae_var(
        V=4096, Cvae=32, ch=160, share_quant_resi=4,    # hard-coded VQVAE hyperparameters
        device=device, patch_nums=patch_nums,
        num_classes=1000, depth=MODEL_DEPTH, shared_aln=False,
    )

# load checkpoints
vae.load_state_dict(torch.load(vae_ckpt, map_location='cpu'), strict=True)
var.load_state_dict(torch.load(var_ckpt, map_location='cpu'), strict=True)
vae.eval(), var.eval()
for p in vae.parameters(): p.requires_grad_(False)
for p in var.parameters(): p.requires_grad_(False)
print(f'prepare finished.')


[constructor]  ==== flash_if_available=True (0/16), fused_if_available=True (fusing_add_ln=0/16, fusing_mlp=0/16) ==== 
    [VAR config ] embed_dim=1024, num_heads=16, depth=16, mlp_ratio=4.0
    [drop ratios ] drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0666667 (tensor([0.0000, 0.0044, 0.0089, 0.0133, 0.0178, 0.0222, 0.0267, 0.0311, 0.0356,
        0.0400, 0.0444, 0.0489, 0.0533, 0.0578, 0.0622, 0.0667]))

[init_weights] VAR with init_std=0.0180422
prepare finished.


In [2]:
############################# 2. Sample with classifier-free guidance

# set args
seed = 0 #@param {type:"number"}
torch.manual_seed(seed)
num_sampling_steps = 250 #@param {type:"slider", min:0, max:1000, step:1} # IRRELEVANT -- variable not used
cfg = 4 #@param {type:"slider", min:1, max:10, step:0.1}
class_labels = (980, 980, 437, 437, 22, 22, 562, 562)  #@param {type:"raw"}
more_smooth = False # True for more smooth output

# seed
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# run faster
tf32 = True
torch.backends.cudnn.allow_tf32 = bool(tf32)
torch.backends.cuda.matmul.allow_tf32 = bool(tf32)
torch.set_float32_matmul_precision('high' if tf32 else 'highest')

# sample
B = len(class_labels)
label_B: torch.LongTensor = torch.tensor(class_labels, device=device)
# with torch.inference_mode():
#     with torch.autocast('cuda', enabled=True, dtype=torch.float16, cache_enabled=True):    # using bfloat16 can be faster
#         recon_B3HW = var.autoregressive_infer_cfg(B=B, label_B=label_B, cfg=cfg, top_k=900, top_p=0.95, g_seed=seed, more_smooth=more_smooth)

# chw = torchvision.utils.make_grid(recon_B3HW, nrow=8, padding=0, pad_value=1.0)
# chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
# chw = PImage.fromarray(chw.astype(np.uint8))
# chw.show()


In [3]:
# chw = torchvision.utils.make_grid(recon_B3HW, nrow=8, padding=0, pad_value=1.0)
# chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
# chw = PImage.fromarray(chw.astype(np.uint8))
# chw.show()
# recon_B3HW.

In [4]:
# optimized_image = torch.rand_like(recon_B3HW, requires_grad=True)
# image_pyramid = vae.img_to_idxBl(optimized_image, patch_nums)

# image_pyramid

In [5]:
# reconstructed = vae.idxBl_to_img(image_pyramid, same_shape=False)

In [6]:
# reconstructed[6].shape

### Encoding the input 

In [7]:
# x = recon_B3HW
x = torch.rand(8, 3, 256, 256)

f = vae.quant_conv(vae.encoder(x))

# ls_f_hat_BChw = vae.quantize.f_to_idxBl_or_fhat(f, to_fhat=False, v_patch_nums=patch_nums)

In [8]:
from torch.nn import functional as F

patch_nums = (1, 2, 3, 4, 5, 6, 8, 10, 13, 16)
coarsness_step = 4

quant_pyramid, f_hat = vae.quantize.f_to_quant_pyramid_and_f_hat(f, patch_nums, coarsness_step)

residual = f - f_hat

In [9]:
len(quant_pyramid)

4

### Using the pyramid as the input for transformer

In [12]:
predicted_residual = var.autoregressive_single_step_prediction(
    quant_pyramid=quant_pyramid, 
    f_hat=f_hat, 
    label_B=label_B, 
    cfg=cfg, 
    top_k=900, 
    top_p=0.95)

In [13]:
f.shape

torch.Size([8, 32, 16, 16])

In [60]:
predicted_residual.shape

torch.Size([8, 32, 16, 16])

### Optimisation POC in latent space

In [14]:
def get_optimisation_loss(f, coarsness_step):
    quant_pyramid, f_hat = vae.quantize.f_to_quant_pyramid_and_f_hat(f, patch_nums, coarsness_step)

    f_residual = f - f_hat

    predicted_residual = var.autoregressive_single_step_prediction(
        quant_pyramid=quant_pyramid, 
        f_hat=f_hat, 
        label_B=label_B, 
        cfg=cfg, 
        top_k=900, 
        top_p=0.95)

    return F.mse_loss(predicted_residual, f_residual)


In [57]:

f = torch.rand([2, 32, 16, 16], requires_grad=True)

class_labels = (22, 562)  #@param {type:"raw"}
label_B: torch.LongTensor = torch.tensor(class_labels, device=device)

optimizer = torch.optim.Adam([f], lr=0.1)

steps_per_coarsness = 100

for coarsness_step in range(1, len((patch_nums)) - 1):
    for i in range(steps_per_coarsness):
        optimizer.zero_grad()
        loss = get_optimisation_loss(f, coarsness_step)
        loss*= patch_nums[coarsness_step]**2 / patch_nums[len(patch_nums)-1]**2
        loss*= (steps_per_coarsness - i) / steps_per_coarsness

        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f'coarsness_step: {coarsness_step}, loss: {loss.item()}')

coarsness_step: 1, loss: 0.004371508955955505
coarsness_step: 1, loss: 0.0003481769817881286
coarsness_step: 1, loss: 0.00036811825702898204
coarsness_step: 1, loss: 0.00014163559535518289
coarsness_step: 1, loss: 0.00015670272114221007
coarsness_step: 1, loss: 0.00011510938202263787
coarsness_step: 1, loss: 9.264543768949807e-05
coarsness_step: 1, loss: 6.400592974387109e-05
coarsness_step: 1, loss: 3.6390963941812515e-05
coarsness_step: 1, loss: 1.6867006706888787e-05
coarsness_step: 2, loss: 0.0005777326296083629
coarsness_step: 2, loss: 0.0005680759786628187
coarsness_step: 2, loss: 0.0006122048944234848
coarsness_step: 2, loss: 0.0005135508254170418
coarsness_step: 2, loss: 0.0003322355914860964
coarsness_step: 2, loss: 0.0001490399445174262
coarsness_step: 2, loss: 0.0001059482601704076
coarsness_step: 2, loss: 5.8833087678067386e-05
coarsness_step: 2, loss: 4.348806032794528e-05
coarsness_step: 2, loss: 1.7639913494349457e-05
coarsness_step: 3, loss: 0.00029077508952468634
coars

In [58]:
image = vae.fhat_to_img(f).add_(1).mul_(0.5)

In [59]:
chw = torchvision.utils.make_grid(image, nrow=8, padding=0, pad_value=1.0)
chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
chw = PImage.fromarray(chw.astype(np.uint8))
chw.show()

Python(9189) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(9190) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


## Random coarsness step


In [47]:
from tqdm import tqdm
f = torch.randn([8, 32, 16, 16], requires_grad=True) 

class_labels = (980, 980, 437, 437, 22, 22, 562, 562)  #@param {type:"raw"}
label_B: torch.LongTensor = torch.tensor(class_labels, device=device)

optimizer = torch.optim.Adam([f], lr=0.1)
total_steps = 400

for i in tqdm(range(total_steps), total=total_steps):
    coarsness_step = np.random.randint(1, len((patch_nums)) - 1)

    optimizer.zero_grad()
    loss = get_optimisation_loss(f, coarsness_step)

    loss*= patch_nums[coarsness_step]**2 / patch_nums[len(patch_nums)-1]**2
    loss*= (total_steps - i) / total_steps # linearly decrease the loss weight
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(f'coarsness_step: {coarsness_step}, loss: {loss.item()}')

  0%|          | 1/400 [00:05<34:39,  5.21s/it]

coarsness_step: 4, loss: 0.09834317117929459


  3%|▎         | 11/400 [03:48<7:03:56, 65.39s/it]

coarsness_step: 7, loss: 0.10095315426588058


  5%|▌         | 21/400 [04:00<19:12,  3.04s/it]  

...

 95%|█████████▌| 381/400 [26:15<00:23,  1.26s/it]

coarsness_step: 4, loss: 0.0003595100424718112


 98%|█████████▊| 391/400 [26:28<00:09,  1.05s/it]

coarsness_step: 4, loss: 0.00018257004558108747


100%|██████████| 400/400 [26:41<00:00,  4.00s/it]


In [48]:
image = vae.fhat_to_img(f).add_(1).mul_(0.5)

In [49]:
chw = torchvision.utils.make_grid(image, nrow=8, padding=0, pad_value=1.0)
chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
chw = PImage.fromarray(chw.astype(np.uint8))
chw.show()

Python(92571) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(92584) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
